In [48]:
# Add all necessary imports and installs
!pip install yfinance pandas numpy matplotlib pytz datetime
import math, numpy as np, pandas as pd, yfinance as yf, datetime, pytz, time, os, warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [49]:
# All parameters
TICKER = "TSLA"
INTERVAL = "5m"
LOOKBACK_DAYS = 60

MARKET_TZ = "US/Eastern"
MARKET_OPEN= "9:30:00"
MARKET_CLOSE= "16:00:00"

VWAP_TOLERANCE = 0.005
STOP_PCT = 0.003
TAKE_MULT = 2.0
INITIAL_CAP = 100000
TOUCH_ATR_MULT = 0.20
EMA_CONFLUENCE = False
VOL_CONFIRM = 0.7
RISK_PCT = 0.01
STRETCH_Z_LONG = -0.6
STRETCH_Z_SHORT = +0.6
ATR_WINDOW = 14
MAX_DAILY_LOSS = 0.03
SLIPPAGE_RATE = 0.0002

In [50]:
# Load all data
end_date = datetime.now()
start_date = end_date - timedelta(days=LOOKBACK_DAYS)

raw = yf.download(TICKER, start=start_date, end=end_date, interval=INTERVAL, auto_adjust=False, progress=False)
if isinstance(raw.columns, pd.MultiIndex):
    raw.columns = raw.columns.droplevel(1)
df = raw.dropna().copy()

# Restrict to Regular Trading Hours
df = df.tz_convert(MARKET_TZ)
df = df.between_time(MARKET_OPEN, MARKET_CLOSE)

In [51]:
def _session_id(index, tz=MARKET_TZ):
    idx = index.tz_convert(tz)
    return pd.Series(idx.date, index=index)

def compute_session_vwap(ohlc):
    g = _session_id(ohlc.index)
    tp = (ohlc["High"] + ohlc["Low"] + ohlc["Close"]) / 3.0
    dv = tp * ohlc["Volume"]
    c_dv = dv.groupby(g).cumsum()
    c_vol = ohlc["Volume"].groupby(g).cumsum().replace(0, np.nan)
    return c_dv / c_vol

def compute_indicators(ohlc):
    out = ohlc.copy()
    out["VWAP"] = compute_session_vwap(ohlc)
    out["EMA20"] = out["Close"].ewm(span=20, adjust=False).mean()
    # ATR
    tr = np.maximum(out["High"]-out["Low"],
                    np.maximum((out["High"]-out["Close"].shift()).abs(),
                               (out["Low"]-out["Close"].shift()).abs()))
    out["ATR"] = tr.rolling(ATR_WINDOW).mean()
    # Relative volume
    out["VolRatio"] = out["Volume"] / (out["Volume"].rolling(50).mean() + 1e-9)
    # Deviation + z-score
    out["Dev"] = out["Close"] - out["VWAP"]
    out["DevZ"] = (out["Dev"] - out["Dev"].rolling(50).mean()) / (out["Dev"].rolling(50).std(ddof=0) + 1e-9)
    return out.dropna()

feat = compute_indicators(df)

In [52]:
def detect_vwap_bounce_signals(f):
    f = f.copy()
    # Two proximity notions, percentage tolerance & ATR-based band
    pct_band = (f["Close"] - f["VWAP"]).abs() / f["VWAP"] <= VWAP_TOLERANCE
    atr_band = (f["Close"] - f["VWAP"]).abs() <= (TOUCH_ATR_MULT * f["ATR"].fillna(f["Close"]*STOP_PCT))
    near_vwap = pct_band | atr_band

    # Rejection/bounce confirmation candle
    long_reject  = (f["Low"] <= f["VWAP"]) & (f["Close"] >= f["VWAP"]) & (f["Close"] > f["Close"].shift(1))
    short_reject = (f["High"] >= f["VWAP"]) & (f["Close"] <= f["VWAP"]) & (f["Close"] < f["Close"].shift(1))

    # Prior stretch context
    stretched_down = f["DevZ"].rolling(30).min() <= STRETCH_Z_LONG
    stretched_up   = f["DevZ"].rolling(30).max() >= STRETCH_Z_SHORT

    # Volume check
    vol_ok = f["VolRatio"] >= VOL_CONFIRM

    # EMA confluence
    if EMA_CONFLUENCE:
        ema_long_ok  = f["Close"] >= f["EMA20"]
        ema_short_ok = f["Close"] <= f["EMA20"]
    else:
        ema_long_ok = ema_short_ok = pd.Series(True, index=f.index)

    # Entries trigger at next bar open, so compute on signal bar
    f["LongSignal"]  = stretched_down & near_vwap & long_reject  & vol_ok & ema_long_ok
    f["ShortSignal"] = stretched_up   & near_vwap & short_reject & vol_ok & ema_short_ok

    # Rare simultaneous signals
    both = f["LongSignal"] & f["ShortSignal"]
    f.loc[both & (f["Dev"] > 0), "LongSignal"] = False
    f.loc[both & (f["Dev"] < 0), "ShortSignal"] = False

    return f

feat = detect_vwap_bounce_signals(feat)

In [53]:
def backtest(f,
             rr=TAKE_MULT,
             slippage=SLIPPAGE_RATE,
             risk_pct=RISK_PCT,
             max_daily_loss=MAX_DAILY_LOSS):
    f = f.copy()
    f["Date"] = f.index.tz_convert(MARKET_TZ).date
    equity = INITIAL_CAP
    daily_pnl = 0.0
    cur_day = None

    in_trade = False
    side = 0        # +1 long, -1 short
    entry_px = np.nan
    stop_px  = np.nan
    take_px  = np.nan
    size     = 0
    trade_id = 0
    bars_in_trade = 0

    blotter = []

    for i in range(1, len(f)):
        ts = f.index[i]
        d  = f.at[ts, "Date"]
        o,h,l,c = f.at[ts, "Open"], f.at[ts, "High"], f.at[ts, "Low"], f.at[ts, "Close"]
        atr = f.at[f.index[i-1], "ATR"] if not np.isnan(f.at[f.index[i-1], "ATR"]) else f.at[f.index[i-1], "Close"]*STOP_PCT

        # New day, reset per day limits
        if (cur_day is None) or (d != cur_day):
            cur_day = d
            daily_pnl = 0.0

        # Manage open trade
        if in_trade:
            bars_in_trade += 1
            hit = None
            # Intrabar stop/take logic
            if side == +1:
                if l <= stop_px:            # stop first
                    fill = stop_px * (1 - slippage)
                    pnl  = (fill - entry_px) * size
                    hit  = ("STOP", fill, pnl)
                elif h >= take_px:
                    fill = take_px * (1 - slippage)
                    pnl  = (fill - entry_px) * size
                    hit  = ("TAKE", fill, pnl)
            else:
                if h >= stop_px:
                    fill = stop_px * (1 + slippage)
                    pnl  = (entry_px - fill) * size
                    hit  = ("STOP", fill, pnl)
                elif l <= take_px:
                    fill = take_px * (1 + slippage)
                    pnl  = (entry_px - fill) * size
                    hit  = ("TAKE", fill, pnl)

            # Timeout at bar close
            if hit is None:
                pass

            if hit is not None:
                reason, exit_px, pnl = hit
                equity += pnl
                daily_pnl += pnl
                blotter.append(dict(
                    trade_id=trade_id, exit_time=ts, exit_px=exit_px, reason=reason, pnl=pnl,
                    side="LONG" if side==+1 else "SHORT"
                ))
                in_trade = False
                side = 0
                size = 0
                entry_px = stop_px = take_px = np.nan

        # Entry, use previous bar signal to enter this bar open
        if not in_trade and abs(daily_pnl) / max(INITIAL_CAP, 1) < max_daily_loss:
            prev = f.index[i-1]
            go_long  = bool(f.at[prev, "LongSignal"])
            go_short = bool(f.at[prev, "ShortSignal"])

            if go_long or go_short:
                side = +1 if go_long else -1
                # Enter at this bar's open with slippage
                entry_px = o * (1 + slippage if side==+1 else 1 - slippage)

                # Stop via ATR distance, fallback to pct if ATR 0
                stop_dist = max(atr, f.at[prev, "Close"]*STOP_PCT)
                if side == +1:
                    stop_px = entry_px - stop_dist
                    take_px = entry_px + rr * (entry_px - stop_px)
                else:
                    stop_px = entry_px + stop_dist
                    take_px = entry_px - rr * (stop_px - entry_px)

                # Position sizing, risk = RISK_PCT * equity
                risk_dollars = equity * risk_pct
                per_share_risk = (entry_px - stop_px) if side==+1 else (stop_px - entry_px)
                size = max(int(risk_dollars / max(per_share_risk, 1e-6)), 0)

                if size > 0:
                    trade_id += 1
                    in_trade = True
                    bars_in_trade = 0
                    blotter.append(dict(
                        trade_id=trade_id, entry_time=ts, entry_px=entry_px, side="LONG" if side==+1 else "SHORT",
                        stop_px=stop_px, take_px=take_px, size=size, day=str(cur_day)
                    ))
                else:
                    # skip if sizing too small
                    in_trade = False
                    side = 0
                    entry_px = stop_px = take_px = np.nan

        f.at[ts, "Equity"] = equity
        f.at[ts, "DailyPnL"] = daily_pnl

    blotter_df = pd.DataFrame(blotter)

    # Ensure the pnl column exists even if no exits yet
    if "pnl" not in blotter_df.columns:
        blotter_df["pnl"] = np.nan

    # Build trade level PnL summary
    exits = blotter_df.dropna(subset=["pnl"])
    if exits.empty:
        perf = {
            "trades": 0,
            "win_rate": np.nan,
            "avg_win": 0.0,
            "avg_loss": 0.0,
            "expectancy": 0.0,
            "cum_pnl": 0.0
        }
    else:
        perf = {
            "trades": int(exits.shape[0]),
            "win_rate": float((exits["pnl"] > 0).mean()),
            "avg_win": float(exits.loc[exits["pnl"] > 0, "pnl"].mean()) if (exits["pnl"] > 0).any() else 0.0,
            "avg_loss": float(exits.loc[exits["pnl"] < 0, "pnl"].mean()) if (exits["pnl"] < 0).any() else 0.0,
            "expectancy": float(exits["pnl"].mean()),
            "cum_pnl": float(exits["pnl"].sum())
        }

    return f, blotter_df, perf

bt_df, blotter, stats = backtest(feat)

print("=== Performance Summary ===")
for k,v in stats.items():
    print(f"{k:>12}: {v}")

print("\n=== Last 10 Trades (blotter) ===")
print(blotter.tail(10))

=== Performance Summary ===
      trades: 98
    win_rate: 0.37755102040816324
     avg_win: 1956.5541984791446
    avg_loss: -1069.3876426679217
  expectancy: 73.0597871529095
     cum_pnl: 7159.859140985131

=== Last 10 Trades (blotter) ===
     trade_id                entry_time    entry_px   side     stop_px  \
186        94 2025-09-18 09:35:00-04:00  429.795933   LONG  427.028859   
187        94                       NaT         NaN   LONG         NaN   
188        95 2025-09-18 09:55:00-04:00  428.080594   LONG  425.201594   
189        95                       NaT         NaN   LONG         NaN   
190        96 2025-09-19 09:35:00-04:00  424.882170   LONG  422.826556   
191        96                       NaT         NaN   LONG         NaN   
192        97 2025-09-19 10:45:00-04:00  425.164950  SHORT  427.129005   
193        97                       NaT         NaN  SHORT         NaN   
194        98 2025-09-22 12:15:00-04:00  438.943779  SHORT  440.260884   
195        98    